In [ ]:
import os
import shutil
import cv2
import statistics

from tensorflow.python.keras.preprocessing.image import img_to_array
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

import keras
from keras import models
from keras.layers import Dense, Dropout, Input, Conv2D, MaxPooling2D, Flatten
from keras.models import Model, Sequential

from tensorflow.keras.optimizers import RMSprop

from sklearn.metrics import confusion_matrix
import seaborn as sns

import pandas as pd

In [ ]:
height_list = []
width_list = []

path1 = "../input/vision-verse/data/train/crownandrootrot"
path2 = "../input/vision-verse/data/train/healthywheat"
path3 = "../input/vision-verse/data/train/leafrust"
path4 = "../input/vision-verse/data/train/wheatloosesmut"

for image in os.listdir(path1):
    image_path = path1+"/"+image
    img = Image.open(image_path)
    height_list.append(img.size[0])
    width_list.append(img.size[1])

for image in os.listdir(path2):
    image_path = path2+"/"+image
    img = Image.open(image_path)
    height_list.append(img.size[0])
    width_list.append(img.size[1])
    
for image in os.listdir(path3):
    image_path = path3+"/"+image
    img = Image.open(image_path)
    height_list.append(img.size[0])
    width_list.append(img.size[1])
    
for image in os.listdir(path4):
    image_path = path4+"/"+image
    img = Image.open(image_path)
    height_list.append(img.size[0])
    width_list.append(img.size[1])

In [ ]:
median_height = statistics.median(height_list)
meidan_width = statistics.median(width_list)

print(median_height)
print(meidan_width)

In [ ]:
Img_size = 262

In [ ]:
data = []
labels = []

os.makedirs('./c')
os.makedirs('./h')
os.makedirs('./l')
os.makedirs('./w')

i = 0

for image in os.listdir(path1):
    
    image_path = path1+"/"+image
    image_using_PIL = Image.open(image_path)

    if len(image_using_PIL.split()) != 3:        
        i += 1
    
    else:
    
        labels.append(0) 

        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        img = cv2.resize(img, (Img_size, Img_size),
                   interpolation = cv2.INTER_NEAREST)

        array = img_to_array(img) 
        data.append(array) 


for image in os.listdir(path2):
    
    image_path = path2+"/"+image
    image_using_PIL = Image.open(image_path)

    if len(image_using_PIL.split()) != 3:
        i += 1
    
    else:
    
        labels.append(1) 

        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        img = cv2.resize(img, (Img_size, Img_size),
                   interpolation = cv2.INTER_NEAREST)

        array = img_to_array(img) 
        data.append(array) 

        
for image in os.listdir(path3):
    
    image_path = path3+"/"+image
    image_using_PIL = Image.open(image_path)

    if len(image_using_PIL.split()) != 3:
        i += 1
    
    else:
    
        labels.append(2) 

        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        img = cv2.resize(img, (Img_size, Img_size),
                   interpolation = cv2.INTER_NEAREST)

        array = img_to_array(img) 
        data.append(array) 
        
for image in os.listdir(path4):
    
    image_path = path4+"/"+image
    image_using_PIL = Image.open(image_path)

    if len(image_using_PIL.split()) != 3:
        i += 1
    
    else:
    
        labels.append(3) 

        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        img = cv2.resize(img, (Img_size, Img_size),
                   interpolation = cv2.INTER_NEAREST)

        array = img_to_array(img) 
        data.append(array) 
        
print(i)

In [ ]:
if (os.path.exists('./c') == True):
    shutil.rmtree('./c')

if (os.path.exists('./h') == True):
    shutil.rmtree('./h')
    
if (os.path.exists('./l') == True):
    shutil.rmtree('./l')
    
if (os.path.exists('./w') == True):
    shutil.rmtree('./w')
    

In [ ]:
print(len(data))
print(len(labels))
data=np.array(data)
labels=np.array(labels)
print(data.shape,labels.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, 
                                                    random_state=100)

In [ ]:
y_train = to_categorical(y_train, 4)
y_test = to_categorical(y_test, 4)

X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

print(X_train.shape, y_train.shape)

In [ ]:
model = Sequential([
    
    Conv2D(50,3, activation='relu', input_shape=(Img_size,Img_size,3)),
  
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    
    Dropout(0.5),
    
    Dense(50, activation='relu'),
    
    Dense(4,activation='softmax') 
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='CategoricalCrossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),steps_per_epoch = 8, 
                    epochs = 50)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, label='training accuracy')
plt.plot(epochs, val_acc, label='validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, label='training loss')
plt.plot(epochs, val_loss, label='validation loss')
plt.title('Training and validation loss')
plt.legend()

In [ ]:
model.save('./deeper_CNN_model.h5')

In [ ]:
model_ = keras.models.load_model('./deeper_CNN_model.h5')

In [ ]:
y_test_pred = model.predict(X_test)

In [ ]:
y_test_pred_list = []

for prediction in y_test_pred:
    y_test_pred_list.append(np.argmax(prediction, axis=None, out=None))
    
print(len(y_test_pred_list))

In [ ]:
y_test_list = []

for prediction in y_test:
    y_test_list.append(np.argmax(prediction, axis=None, out=None))
    
print(len(y_test_list))

In [ ]:
matrix = confusion_matrix(y_test_list,y_test_pred_list)

print(matrix)

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

accuracy = accuracy_score(y_test_list,y_test_pred_list)
print('Accuracy: %f' % accuracy)

precision = precision_score(y_test_list,y_test_pred_list, average='weighted')
print('Precision: %f' % precision)

recall = recall_score(y_test_list,y_test_pred_list, average='weighted')
print('Recall: %f' % recall)

f1 = f1_score(y_test_list,y_test_pred_list, average='weighted')
print('F1 score: %f' % f1)

In [ ]:
def prediction(_path):
    image = Image.open(_path)
    if len(image.split()) != 3:
        result = 1
    else:
        image = image.resize((Img_size, Img_size))
        image = np.reshape(image, (1, Img_size, Img_size, 3))
        image = image/255.0
        result = model_.predict(image)
        result = np.argmax(result, axis=1)[0]
    return result

print(prediction('../input/vision-verse/data/train/healthywheat/00071.jpg'))

In [ ]:
test = pd.read_csv('../input/vision-verse/data/sample_submission.csv')
print(test.shape)
print(test.head())

In [ ]:
path_ = '../input/vision-verse'

for i in range(0,1101):
    
    image_path = test.iloc[i,0]
    image = path_ + '/' + image_path
    
    pred = prediction(image)
    
    if pred == 0:
        test.iloc[i,1] = 'crownandrootrot'
    elif pred == 1:
        test.iloc[i,1] = 'healthywheat'
    elif pred == 2:
        test.iloc[i,1] = 'leafrust'
    else:
        test.iloc[i,1] = 'wheatloosesmut'

In [ ]:
test.to_csv('./Predictions.csv',index=False)